# Week 2 Practice Gradio by Creating Brochure

- **Author**: [stoneskin](https://www.github.com/stoneskin)

## Implementation

- Use OpenRouter.ai for all different types of LLM models, include many free models from Google,Meta and Deepseek

Full code for the Week2 Gradio practice is below:

In [0]:
import os
import json
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr 

load_dotenv(override=True)

api_key = os.getenv('Open_Router_Key')
if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
   print("API key looks good so far")
else:
   print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
  
  
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

MODEL_Gemini2FlashThink = 'google/gemini-2.0-flash-thinking-exp:free'
MODEL_Gemini2Pro ='google/gemini-2.0-pro-exp-02-05:free'
MODEL_Gemini2FlashLite = 'google/gemini-2.0-flash-lite-preview-02-05:free'
MODEL_Meta_Llama33 ='meta-llama/llama-3.3-70b-instruct:free'
MODEL_Deepseek_V3='deepseek/deepseek-chat:free'
MODEL_Deepseek_R1='deepseek/deepseek-r1-distill-llama-70b:free'
MODEL_Qwen_vlplus='qwen/qwen-vl-plus:free'
MODEL_OpenAi_o3mini = 'openai/o3-mini'
MODEL_OpenAi_4o = 'openai/gpt-4o-2024-11-20'
MODEL_Claude_Haiku = 'anthropic/claude-3.5-haiku-20241022'




        


In [0]:
MODEL=MODEL_Gemini2Pro  # choice the model you want to use

####################
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

##############################
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

##############################
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

##############################
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    
##############################
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

##############################
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    print("get_links:", result)
    return json.loads(result)

##############################
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

##############################
def get_all_details(url):
    print("get_all_details:", url)   
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [0]:
########### modified stream brochure function for gradio ###################
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [0]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="company Name"), gr.Textbox(label="URL")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()